In [5]:
# import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [6]:
data=[]

def kargo(notice): # Is shipping free or not?
    try:
     a= str(notice.find("div",class_="stmp fc")) 
     return a[21:33]
    except AttributeError:
        return("KARGO BEDAVA DEĞİL")

def product(notice): # Product name
    try:
     notice = notice.find("span", class_="prdct-desc-cntnr-name hasRatings").text
     return notice
    except AttributeError:
        return("Faulty")

def rating(notice): # How many people rated this product?
    try:

     notice = notice.find("span", class_="ratingCount").text.replace("(","").replace(")","")
     return notice
    except AttributeError:
        return("Faulty")

In [7]:
for page in range (1,99):

    url = "https://www.trendyol.com/sr?fl=sepetteurunler&sst=BEST_SELLER&pi=" + str(page) # url of the website
    headers = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
    page = requests.get(url, headers=headers).text # connect to website
    soup = BeautifulSoup(page,"html.parser") # raw data
    notices = soup.find_all("div",class_="p-card-wrppr with-campaign-view") # filtering the data

    for notice in notices:
        item={}
        item["brand"] = notice.find("span", class_="prdct-desc-cntnr-ttl").text # Supplier of the product
        item["product"] = product(notice) # Product name
        item["rating"] = rating(notice) # The number of people who rated the product
        stars = notice.find_all('div',attrs={'class':'full'})
        star_rating=0
        # print(url) # activate this code to see the current page the program working on.

        for i in stars: 
            style = i['style']
            style = int(style.split(':',1)[1].split(";")[0].replace("%",""))/100 # in the source code %100 = 1 star. so we get rid of the % sign and divided it by 100 to see how many starts the product has.
            star_rating= star_rating + style
        item["star_rating"] = star_rating
        item["price"] = notice.find("div", class_="prc-box-dscntd").text.replace(" TL","")
        item["shipment"] = kargo(notice)
        data.append(item)

In [8]:
# saving the data as xlsx and csv
df = pd.DataFrame(data)
df.to_excel("trendyol.xlsx")
df.to_csv("trendyol.csv")